Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [ ]:
# %matplotlib inline
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

# Chapter 2 - Data Preprocessing

## 2.1 Converting prices to returns

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd 
import numpy as np
import yfinance as yf

2. Download the data and keep the adjusted close prices only:

In [ ]:
df = yf.download("AAPL", 
                 start="2010-01-01", 
                 end="2020-12-31",
                 progress=False)

df = df.loc[:, ["Adj Close"]]

3. Convert adjusted close prices to simple and log returns:

In [ ]:
df["simple_rtn"] = df["Adj Close"].pct_change()
df["log_rtn"] = np.log(df["Adj Close"]/df["Adj Close"].shift(1))

4. Inspect the output:

In [ ]:
df.head()

## 2.2 Adjusting the returns for inflation

### How to do it...

0. Recreating the steps from the previous recipe:

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip install cpi

In [ ]:
import yfinance as yf

df = yf.download("AAPL", 
                 start="2010-01-01", 
                 end="2020-12-31",
                 progress=False)

df = df.loc[:, ["Close"]]

1. Import libraries and authenticate:

In [ ]:
import pandas_datareader.data as web

# Download CPI data from FRED
df_cpi = web.DataReader("CPIAUCSL", "fred",
                        start="2009-12-01",
                        end="2020-12-31")

# Rename the column for consistency
df_cpi = df_cpi.rename(columns={"CPIAUCSL": "cpi"})

df_cpi.head()

2. Resample daily prices to monthly:

In [ ]:
df = df.resample("M").last()
df

3. Download inflation data from Nasdaq Data Link:

In [ ]:
df_cpi = (
    nasdaqdatalink.get(dataset="RATEINF/CPI_USA", 
                       start_date="2009-12-01", 
                       end_date="2020-12-31")
    .rename(columns={"Value": "cpi"})
)

In [ ]:
df_cpi

4. Join inflation data to prices:

In [ ]:
df = df.join(df_cpi, how="left")

5. Calculate simple returns and inflation rate:

In [ ]:
df["simple_rtn"] = df["Adj Close"].pct_change()
df["inflation_rate"] = df["cpi"].pct_change()

6. Adjust the returns for inflation:

In [ ]:
df["real_rtn"] = (
    (df["simple_rtn"] + 1) / (df["inflation_rate"] + 1) - 1
)
df.head()

### There's more

1. Import the library:

In [ ]:
import cpi

# This will download the latest data from the Bureau of Labor Statistics
cpi.update()

In [ ]:
# in the case of seeing the `StaleDataWarning: CPI data is out of date`
cpi.update()

2. Obtain the default CPI series:

In [ ]:
cpi_series = cpi.series.get()
cpi_series

In [ ]:
print(cpi_series)

3. Convert the object into a `pandas` DataFrame:

In [ ]:
df_cpi_2 = cpi_series.to_dataframe()

4. Filter the DataFrame and view the top 12 observations:

In [ ]:
df_cpi_2.query("period_type == 'monthly' and year >= 2010") \
        .loc[:, ["date", "value"]] \
        .set_index("date") \
        .head(12)

## 2.3 Changing the frequency of time series data

### How to do it...

0. Obtain the log returns in case of starting in this recipe:

In [ ]:
import pandas as pd 
import yfinance as yf
import numpy as np

# download data 
df = yf.download("AAPL", 
                 start="2000-01-01", 
                 end="2010-12-31", 
                 auto_adjust=False,
                 progress=False)

# keep only the adjusted close price
df = df.loc[:, ["Adj Close"]] \
       .rename(columns={"Adj Close": "adj_close"})

# calculate simple returns
df["log_rtn"] = np.log(df["adj_close"]/df["adj_close"].shift(1))

# remove redundant data
df = df.drop("adj_close", axis=1) \
       .dropna(axis=0)

df.head()

1. Import the libraries:

In [ ]:
import pandas as pd 
import numpy as np

2. Define the function for calculating the realized volatility:

In [ ]:
def realized_volatility(x):
    return np.sqrt(np.sum(x**2))

3. Calculate monthly realized volatility:

In [ ]:
df_rv = (
    df.groupby(pd.Grouper(freq="M"))
    .apply(realized_volatility)
    .rename(columns={"log_rtn": "rv"})
)

4. Annualize the values:

In [ ]:
df_rv.rv = df_rv["rv"] * np.sqrt(12)

5. Plot the results:

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(df)
ax[0].set_title("Apple's log returns (2000-2012)")
ax[1].plot(df_rv)
ax[1].set_title("Annualized realized volatility")

sns.despine()
plt.tight_layout()
# plt.savefig('images/figure_2_5', dpi=200)

## 2.4 Different ways of imputing missing data

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import numpy as np
import nasdaqdatalink

2. Download the inflation data from Nasdaq Data Link:

In [ ]:
import pandas_datareader.data as web

df = web.DataReader("CPIAUCSL", "fred",
                    start="2015-01-01",
                    end="2020-12-31").rename(columns={"CPIAUCSL": "cpi"})

3. Introduce 5 missing values at random:

In [ ]:
np.random.seed(42)
rand_indices = np.random.choice(df.index, 5, replace=False)

df["cpi_missing"] = df.loc[:, "cpi"]
df.loc[rand_indices, "cpi_missing"] = np.nan
df.head()

4. Fill the missing values using different methods:

In [ ]:
# Use the modern .bfill() and .ffill() methods
df["method_bfill"] = df[["cpi_missing"]].bfill()
df["method_ffill"] = df[["cpi_missing"]].ffill()

5. Inspect the results by displaying the rows in which we created the missing values:

In [ ]:
df.loc[rand_indices].sort_index()

6. Plot the results for years 2015-2016:

In [ ]:
df.loc[:"2017-01-01"] \
  .drop(columns=["cpi_missing"]) \
  .plot(title="Different ways of filling missing values");

sns.despine()
plt.tight_layout()
# plt.savefig('images/figure_2_8', dpi=200)

### There's more

1. Use linear interpolation to fill the missing values:

In [ ]:
df["method_interpolate"] = df[["cpi_missing"]].interpolate()

2. Inspect the results:

In [ ]:
df.loc[rand_indices].sort_index()

3. Plot the results:

In [ ]:
df.loc[:"2017-01-01"] \
  .drop(columns=["cpi_missing"]) \
  .plot(title="Different ways of filling missing values");

sns.despine()
plt.tight_layout()
# plt.savefig('images/figure_2_10', dpi=200)

## 2.5 Converting currencies

### How to do it...

1. Import the libraries:

In [ ]:
!pip install forex-python

In [ ]:
import pandas as pd 
import yfinance as yf
from forex_python.converter import CurrencyRates


2. Download Apple's OHLC prices from January 2020:

In [ ]:
df = yf.download("AAPL", 
                 start="2020-01-01", 
                 end="2020-01-31",
                 progress=False)
df = df.drop(columns=["Adj Close", "Volume"])

3. Instantiate the `CurrencyRates` object:

In [ ]:
c = CurrencyRates()

4. Download the USD/EUR rate for each required date:

In [ ]:
# Download the EUR/USD exchange rate data for the same period
df_rates = yf.download("EURUSD=X",
                       start="2020-01-01",
                       end="2020-01-31",
                       progress=False)

# Keep only the closing price and rename it
df_rates = df_rates[["Close"]].rename(columns={"Close": "usd_eur"})

# Join the exchange rates to the AAPL price data
df = df.join(df_rates, how="left").ffill() # forward-fill any missing rates

# Convert the price columns to EUR
for column in ['Open', 'High', 'Low', 'Close']:
    df[f"{column}_EUR"] = df[column] * df["usd_eur"]

df.head().round(3)

5. Convert the prices in USD to EUR:

In [ ]:
for column in df.columns[:-1]:
    df[f"{column}_EUR"] = df[column] * df["usd_eur"]
df.head().round(3)

### There's more

1. Get the USD exchange rates to 31 available currencies:

In [ ]:
usd_rates = c.get_rates("USD")
usd_rates

In [ ]:
len(usd_rates)

2. Download the USD/EUR exchange rate from Yahoo Finance:

In [ ]:
df = yf.download("USDEUR=X", 
                 start="2000-01-01", 
                 end="2010-12-31",
                 progress=False)
df.head()

## 2.6 Different ways of aggregating trade data

### How to do it...

1. Import the libraries:

In [ ]:
!pip install python-binance

In [ ]:
from binance.spot import Spot as Client
import pandas as pd 
import numpy as np


2. Instantiate the Binance client and download the last 500 `BTCEUR` trades:

In [ ]:
spot_client = Client(base_url="https://api3.binance.com")
r = spot_client.trades("BTCEUR")

3. Process the downloaded trades into a `pandas` DataFrame:

In [ ]:
df = (
    pd.DataFrame(r)
    .drop(columns=["isBuyerMaker", "isBestMatch"])
)
df["time"] = pd.to_datetime(df["time"], unit="ms")

for column in ["price", "qty", "quoteQty"]:
    df[column] = pd.to_numeric(df[column])
df

4. Define a function aggregating the raw trades information:

In [ ]:
def get_bars(df, add_time=False):
    """[summary]

    Args:
        df ([type]): [description]

    Returns:
        [type]: [description]
    """
    ohlc = df["price"].ohlc()
    vwap = (
        df.apply(lambda x: np.average(x["price"], weights=x["qty"]))
        .to_frame("vwap")
    )
    vol = df["qty"].sum().to_frame("vol")
    cnt = df["qty"].size().to_frame("cnt")
    
    if add_time:
        time = df["time"].last().to_frame("time")
        res = pd.concat([time, ohlc, vwap, vol, cnt], axis=1)
    else:
        res = pd.concat([ohlc, vwap, vol, cnt], axis=1)
    return res

5. Get time bars:

In [ ]:
df_grouped_time = df.groupby(pd.Grouper(key="time", freq="1Min"))
time_bars = get_bars(df_grouped_time)
time_bars

6. Get tick bars:

In [ ]:
bar_size = 50 
df["tick_group"] = (
    pd.Series(list(range(len(df))))
    .div(bar_size)
    .apply(np.floor)
    .astype(int)
    .values
)
df_grouped_ticks = df.groupby("tick_group")
tick_bars = get_bars(df_grouped_ticks, add_time=True)
tick_bars

7. Get volume bars:

In [ ]:
bar_size = 1 
df["cum_qty"] = df["qty"].cumsum()
df["vol_group"] = (
    df["cum_qty"]
    .div(bar_size)
    .apply(np.floor)
    .astype(int)
    .values
)
df_grouped_ticks = df.groupby("vol_group")
volume_bars = get_bars(df_grouped_ticks, add_time=True)
volume_bars

8. Get dollar bars:

In [ ]:
bar_size = 50000 
df["cum_value"] = df["quoteQty"].cumsum()
df["value_group"] = (
    df["cum_value"]
    .div(bar_size)
    .apply(np.floor)
    .astype(int)
    .values
)
df_grouped_ticks = df.groupby("value_group")
dollar_bars = get_bars(df_grouped_ticks, add_time=True)
dollar_bars